In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Smote_train.csv')

In [3]:
df.columns

Index(['person_age', 'person_income', 'person_home_ownership', 'loan_intent',
       'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income',
       'loan_status'],
      dtype='object')

In [5]:
df.head()

,person_age,person_income,person_home_ownership,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,loan_status
0,37,35000,0,0,0,6000,11.49,0.17,0
1,22,56000,1,1,1,4000,13.35,0.07,0
2,29,28800,1,2,2,6000,8.90,0.21,0
3,30,70000,0,3,0,12000,11.11,0.17,0
4,22,60000,0,1,2,6000,6.92,0.10,0


In [6]:
X_train = df[['person_age', 'person_income', 'person_home_ownership',
         'loan_intent', 'loan_grade', 'loan_amnt',
         'loan_int_rate', 'loan_percent_income']]
y = df['loan_status'].values

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

In [9]:
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.3, random_state=42)

In [22]:
model = Sequential()
model.add(LSTM(65, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

e:\AI_Projek\Compettion_Load_Predict\bakso2\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=80, batch_size=64, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

Epoch 1/80
881/881 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8824 - loss: 0.2770 - val_accuracy: 0.8821 - val_loss: 0.2707
Epoch 2/80
881/881 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8867 - loss: 0.2663 - val_accuracy: 0.8832 - val_loss: 0.2700
Epoch 3/80
881/881 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8842 - loss: 0.2718 - val_accuracy: 0.8840 - val_loss: 0.2696
Epoch 4/80
881/881 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8841 - loss: 0.2718 - val_accuracy: 0.8833 - val_loss: 0.2704
Epoch 5/80
881/881 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8849 - loss: 0.2711 - val_accuracy: 0.8831 - val_loss: 0.2704
Epoch 6/80
881/881 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8859 - loss: 0.2705 - val_accuracy: 0.8826 - val_loss: 0.2700
Epoch 7/80
881/881 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8843 - loss: 0.2705 - val_accuracy: 0.8824 - val_loss: 0.2694
Epoch 8/80
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8871 - loss: 0.2662 - val_accuracy: 0.

In [26]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(X_test)

# Menggunakan threshold 0.5 untuk binarize hasil prediksi
predictions_binary = (predictions > 0.5).astype(int)

# Menghitung classification report
print("Classification Report:")
print(classification_report(y_test, predictions_binary))

# Menghitung confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions_binary))

944/944 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.92      0.89     15172
           1       0.92      0.85      0.88     15005

    accuracy                           0.89     30177
   macro avg       0.89      0.89      0.89     30177
weighted avg       0.89      0.89      0.89     30177

Confusion Matrix:
[[14011  1161]
 [ 2218 12787]]


In [27]:
dft = pd.read_csv('test.csv')

In [28]:
dft['person_home_ownership'] = dft['person_home_ownership'].map({'RENT' : 0, 'OWN': 1, 'MORTGAGE' : 2, 'OTHER' : 3})
dft['loan_intent'] = dft['loan_intent'].map({'EDUCATION' : 0, 'MEDICAL' : 1, 'PERSONAL' : 2, 'VENTURE' : 3, 'DEBTCONSOLIDATION' : 4, 'HOMEIMPROVEMENT' : 5})
dft['loan_grade'] = dft['loan_grade'].map({'B' : 0, 'C' : 1, 'A' : 2, 'D' : 3,'E' : 4, 'F' : 5, 'G' : 6})

In [29]:
X_rl = dft[['person_age', 'person_income', 'person_home_ownership',
                'loan_intent', 'loan_grade', 'loan_amnt',
                'loan_int_rate', 'loan_percent_income']]

In [30]:
X_scaled_rl = scaler.fit_transform(X_rl)
X_reshapedrl = X_scaled_rl.reshape((X_scaled_rl.shape[0], 1, X_scaled_rl.shape[1]))

In [31]:
predictions = model.predict(X_reshapedrl)


1222/1222 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  


In [32]:
predictions_1d = predictions.flatten() 

In [33]:
results = pd.DataFrame({
    'id': dft['id'],
    'loan_status': predictions_1d
})

# Menyimpan hasil ke CSV
results.to_csv('Zoltrak6.csv', index=False)